# Homework: Phrasal Chunking

This is your documentation for the chunker homework.

In [24]:
import perc
import default
import sys
from collections import defaultdict

feat_vec = {}
tagset = []
train_data = []

# The following code was written to implement http://www.cs.sfu.ca/~anoop/papers/pdf/ai05.pdf but it took a very long time
# so we had to drop this implementation.

# def specilization(train_data, tagset, istrain=False):
#     words={}
#     for i in train_data:
#         for j in i[0]:
#             if j in words.keys():
#                 words[j]+=1
#             else:
#                 words[j]=1
#     Ws=list()
#     for key,value in words.items():
#         if value>=100:
#             Ws.append(key)
#         else:
#             h=key.split()
#             if(h[1] in ['NP','VP','PP','ADVP']):
#                 if value>=50:
#                     Ws.append(key)
#     words_list=[]
#     for word in Ws:
#         words_list.append(word.split()[0])
#     new_train=[]
#     for i in train_data:
#         temp_list=[]
#         for j in i[0]:
#             if j.split()[0] in words_list:
#                 temp_list.append(j.split()[0]+' '+j.split()[0]+'-'+j.split()[1]+' '+j.split()[0]+'-'+j.split()[1]+'-'+j.split()[2])
#                 tag=j.split()[0]+'-'+j.split()[1]+'-'+j.split()[2]
#                 if istrain:
#                     if tag not in tagset:
#                         tagset.append(tag)
#             else:
#                 temp_list.append(j.split()[0]+' '+j.split()[1]+' '+j.split()[1]+'-'+j.split()[2])
#                 tag=j.split()[1]+'-'+j.split()[2]
#                 if istrain:
#                     if tag not in tagset:
#                         tagset.append(tag)
#         new_train.append((temp_list,i[1]))
#     return new_train, tagset


def perc_train(train_data, tagset,numepochs):
    feat_vec = defaultdict(int)
    sigma = defaultdict(int)
    mistakes=0
    # For each epoch we go through all the sentences in the training set.
    for epoch in range(numepochs):
        mistakes=0
        # Each sentence is passed through the viterbi to get the argmax output of the labels.
        for sentence in train_data:
            # The prediction from viterbi is stored in pred.
            pred=perc.perc_test(feat_vec,sentence[0],sentence[1],tagset,tagset[0])
            true=[word.split()[2] for word in sentence[0]]
            count=0
            # Each label returned from the result of viterbi is checked with the true label.
            for i in range(len(pred)):
                count+=20
                if(pred[i]!=true[i]): 
                    # Record the mistakes in all epochs
                    mistakes+=1
                    # For features of each of the word in the sentence, we make the updates in weight vector.
                    for j in range(count-20,count):
                        # We give a -1 update to the features of the wrong label.
                        if (sentence[1][j],pred[i]) in feat_vec.keys():
                            feat_vec[(sentence[1][j],pred[i])]-=1
                        else:
                            feat_vec[(sentence[1][j],pred[i])]=-1
                        # We give a +1 update to the features of the true label.
                        if (sentence[1][j],true[i]) in feat_vec.keys():
                            feat_vec[(sentence[1][j],true[i])]+=1
                        else:
                            feat_vec[(sentence[1][j],true[i])]=1
                    if i>0:
                        # Similarly, we give -1 update to the wrong bigram features and 
                        # +1 update to bigram features in true label.
                        if ("B:"+pred[i-1],pred[i]) in feat_vec.keys():
                            feat_vec[("B:"+pred[i-1],pred[i])]-=1
                        else:
                            feat_vec[("B:"+pred[i-1],pred[i])]=-1
                        if ("B:"+true[i-1],true[i]) in feat_vec.keys():
                            feat_vec[("B:"+true[i-1],true[i])]+=1
                        else:
                            feat_vec[("B:"+true[i-1],true[i])]=1
            # After going through each sentence, we aggregate the weights for all the features as mentioned in 
            # http://www.cs.sfu.ca/~anoop/papers/pdf/syntax-parsing-survey-2011.pdf
            for feat,weight in feat_vec.items():
                if feat in sigma.keys():
                    sigma[feat]+=weight
                else:
                    sigma[feat]=weight
                
        print('Mistakes in epoch :',epoch,' are: ',mistakes)
        
    # We average the weight parameter using the formula γ = σ/(mT) mentioned in the above mentioned paper.
    for feat,weight in sigma.items():
        sigma[feat]=weight/(len(train_data)*numepochs)
                       
    # insert your code here
    # please limit the number of iterations of training to n iterations
    return sigma

tagset = perc.read_tagset("data/tagset.txt")
print("reading data ...", file=sys.stderr)
train_data = perc.read_labeled_data("data/train.txt.gz", "data/train.feats.gz", verbose=False)
# train_data, tagset = specilization(train_data, tagset, istrain=True)
print("done.", file=sys.stderr)
feat_vec = perc_train(train_data, tagset,10)
perc.perc_write_to_file(feat_vec, "default.model")
print("wrote model to disk")

reading data ...
done.


Mistakes in epoch : 0  are:  16465
Mistakes in epoch : 1  are:  9087
Mistakes in epoch : 2  are:  5850
Mistakes in epoch : 3  are:  4217
Mistakes in epoch : 4  are:  3144
Mistakes in epoch : 5  are:  2705
Mistakes in epoch : 6  are:  2060
Mistakes in epoch : 7  are:  1652
Mistakes in epoch : 8  are:  1463
Mistakes in epoch : 9  are:  1192
wrote model to disk


# Baseline and Weighted Average Implementation

The above code is an implementation of the baseline, which gave us an F-score of around <b>92.3</b>.
The baseline was implemented by taking reference from <i>Michael Collins. Discriminative Training Methods for Hidden Markov Models: Theory and Experiments with Perceptron Algorithms. EMNLP 2002.</i>

Following this, we implemented the averaged perceptron algorithm. This algorithm was referenced from <i>http://www.cs.sfu.ca/~anoop/papers/pdf/syntax-parsing-survey-2011.pdf</i>. After completing this implementation, we got an <b>F-score</b> of <b>93.61</b> with training on <b>10 iterations</b>. This the current model that we are using.

# Specialized Representations

The team also tried to implement specialized reperesentations for the input. We took reference from <i>http://www.cs.sfu.ca/~anoop/papers/pdf/ai05.pdf</i> in order to tranform the input into new <b>pos tags</b> and <b>chunk labels</b>.

We managed to transform the input and created new chunk tags as mentioned in the paper. We had <b>175</b> words in the <b>Ws</b> set which is used to produce various kinds of lexicalized models.

Also, the tagset increased from <b>22</b> to <b>975</b>. But, we couldn't go through with this implementation because we had limited computing resources and the code took a very long time and kept on running without producing results.

In [ ]:
%%capture --no-stderr output
print("reading test data ...", file=sys.stderr)
test_data = perc.read_labeled_data("data/dev.txt", "data/dev.feats", verbose=False)
print("done.", file=sys.stderr)
feat_vec = perc.perc_read_from_file("default.model")
perc.perc_testall(feat_vec, test_data, tagset)

reading test data ...
done.


In [26]:
import score_chunks
boundary = "-X-" # something to use as boundary between sentences
outside = "O" # tag used to mark the outside of any chunk
conlleval = False # use conlleval (should be False for most use cases)
numfeats = 2 # number of columns to consider as features, typically "word POStag"
(test, _) = score_chunks.readTestFile(str(output), boundary, outside, conlleval, numfeats)
with open("data/reference500.txt") as f:
    (reference, _) = score_chunks.readTestFile(f.read(), boundary, outside, conlleval, numfeats)
print("Score: %.2f" % score_chunks.corpus_fmeasure(reference, test, False))

processed 500 sentences with 10375 tokens and 5783 phrases; found phrases: 5789; correct phrases: 5416
             ADJP: precision:  73.20%; recall:  71.72%; F1:  72.45; found:     97; correct:     99
             ADVP: precision:  78.37%; recall:  80.69%; F1:  79.51; found:    208; correct:    202
            CONJP: precision: 100.00%; recall:  60.00%; F1:  75.00; found:      3; correct:      5
             INTJ: precision:   0.00%; recall:   0.00%; F1:   0.00; found:      0; correct:      1
               NP: precision:  94.33%; recall:  94.05%; F1:  94.19; found:   3017; correct:   3026
               PP: precision:  96.76%; recall:  97.95%; F1:  97.35; found:   1236; correct:   1221
              PRT: precision:  80.95%; recall:  77.27%; F1:  79.07; found:     21; correct:     22
             SBAR: precision:  84.62%; recall:  82.24%; F1:  83.41; found:    104; correct:    107
               VP: precision:  93.56%; recall:  93.82%; F1:  93.69; found:   1103; correct:   1100
accura